In [13]:
from abc import ABC, abstractmethod
from IPython.display import display, clear_output
from typing import List, Any, Optional
import numpy as np 
import pandas as pd 
from pathlib import Path
from lib.logger import log

class DisplayAPI(ABC):
    @abstractmethod
    def display(self):
        pass
    
class Print(DisplayAPI):
    def display(self, some_shit):
        print(some_shit.data)
    
class Display(DisplayAPI):
    def display(self, some_shit):
        display(some_shit.data)
    
class RawData(ABC):
    def __init__(self, file, api: Optional[DisplayAPI] = Display() ) -> None:
        self.file = file
        self.api = api
        
        ###
        self.get_data(file)
     
    def display(self):
        self.api.display(self)
        
    @abstractmethod
    def get_data(self, file):
        pass
    
    @property
    def text(self):
        return self.file.read_text()
    @property
    def lines(self):
        return self.text.splitlines()

    
class CSVFile(RawData):
    def get_data(self, file):
        self.raw_data= '\n'.join(self.get_sections()[-1]).encode()
        self.data = np.frombuffer(self.raw_data)
    
    def get_sections(self) -> List[Any]:
        indexes = [ *[k for k,line in enumerate(self.lines) if line==''], -1]
        return [self.lines[i+1:k] for i,k in zip(indexes[0::2], indexes[1::2])]

class Avantes(RawData):
    def get_data(self, file, header=(5,3)):
        header_line, skip = header
        conv=lambda x: x.replace(',', '.').encode()
        header = self.lines[header_line].split(';')
        data = np.genfromtxt((conv(x) for x in open(file)),skip_header=header_line+skip, skip_footer=0, delimiter=';')
        
        df = pd.DataFrame(data[:,1:], index=data[:,0], columns=header[1:])
        df.index.name=header[0]

        self.data=df
        
class Hitachi(RawData):
    def get_data(self, file, header=(30,1)):
        header_line, skip = header

        header = self.lines[header_line].split('\t')
        data = np.genfromtxt(file,skip_header=header_line+skip,skip_footer=0) 
        
        df = pd.DataFrame(data[:,1:], index=data[:,0], columns=header[1:])
        df.index.name=header[0]

        self.data=df
        
absorbance_csv =  Path('../patka-pomiary/CPE45_PFOBPy_comocat.txt')
fluorescence_csv =  Path('../patka-pomiary/func39.txt')

rd = Avantes( file=fluorescence_csv, api=Display() ) # api=Display() is set by default 
rd2 = Hitachi( file=absorbance_csv, api=Display() )

rd.data
# rd.display()
# rd2.display()

,Sample,Dark,Reference,Scope Corrected for Dark
Wave,,,,
900.12,0.000,0.000,0.000,0.00000
903.33,6707.667,6392.333,6602.000,315.33333
906.55,6594.467,6416.933,6549.600,177.53333
909.76,11168.933,10925.533,11102.867,243.40000
912.97,11050.667,10807.867,10998.467,242.80000
...,...,...,...,...
1692.65,10378.667,10290.600,10303.933,88.06667
1695.73,10633.000,10522.600,10547.733,110.40000
1698.80,9666.400,9549.800,9588.533,116.60000
